In [1]:
#import required functions
import warnings
warnings.filterwarnings("ignore")
import math
import os
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import pylab as plt
import numpy as np
from IPython.display import Image
import matplotlib as matplotlib
import datetime
from pandas import DataFrame
from openpyxl import load_workbook
from openpyxl.drawing.image import Image
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm
import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import itertools
from fbprophet import Prophet
from scipy.interpolate import interp1d
from sklearn.preprocessing import StandardScaler
import xlrd

In [2]:
clist='/Users/JoyThompson/Desktop/Automate CK Monthly Billing/clientlist.xlsx'
outdoorxl='/Users/JoyThompson/Desktop/Automate CK Monthly Billing/outdoorsales.xlsx'
outdoorsheet=pd.read_excel(outdoorxl, sheetname='outdoorhub')
clientlist= pd.read_excel(clist, sheetname='Sheet1')
client=clientlist.set_index('Client Name')
clients=client.index
print(clients)

Index(['3B Medical', 'Ambreesh', 'BHF-USA', 'Brain Armor', 'C4 Belts',
       'Coyuchi', 'Dammit Dolls', 'DaVinci', 'Dedeco', 'Domesis',
       'Fast Growing Trees', 'Outdoor Hub (Five Ten)', 'Forever New',
       'geckobrands', 'Geox', 'Ginsu', 'Indosole', 'Innova', 'Jax Wax',
       'Juggernaut', 'Little Giraffe', 'LivBay Lash (sassy lashes)',
       'Marula Oil', 'MasqueBar', 'Mavi', 'Molex', 'Montana West', 'Nano Film',
       'Not Rated', 'Odd Sox', 'Organyc', 'P.E. Nation', 'Perry Ellis',
       'Pet 'n Shape', 'Peter Grimm', 'Rago', 'Snif Sax', 'Soludos',
       'Spin Life', 'Westmoor', 'Wine Enthusiast', 'Workman's Friend',
       'Worldbook', 'David's Wholesale', 'Knit Rite (Therafirm)',
       'Knit Rite (Therafirm) - Walmart', 'Street Strider',
       'Outdoor Hub (Five Ten)'],
      dtype='object', name='Client Name')


In [3]:
earnlist=[]
for ind in outdoorsheet.index:
    sales=outdoorsheet.loc[ind, 'Shipped Product Sales']
    earn=float(sales*.08)
    if earn <3000:
        earn= 3000
    earnlist.append(earn)
outdoorsheet['Billing']=earnlist
outfc=outdoorsheet.drop(columns=['Shipped Product Sales'], axis=0)

print(outfc)

         Date     Billing
0  2019-01-01   5566.4000
1  2019-02-01   3443.3200
2  2019-03-01   3000.0000
3  2019-04-01   7374.8800
4  2019-05-01   8560.8080
5  2019-06-01   9517.5200
6  2019-07-01  10414.7016
7  2019-08-01   8380.2160
8  2019-09-01   5338.4880
9  2019-10-01   3660.5600
10 2019-11-01   3000.0000
11 2019-12-01  28171.4000


In [4]:
datesall=['2018-11-01','2018-12-01','2019-01-01','2019-02-01','2019-03-01','2019-04-01',
         '2019-05-01','2019-06-01','2019-07-01','2019-08-01','2019-09-01','2019-10-01','2019-11-01','2019-12-01',
         '2020-01-01','2020-02-01','2020-03-01','2020-04-01','2020-05-01','2020-06-01','2020-07-01',
         '2020-08-01','2020-09-01','2020-10-01','2020-11-01','2020-12-01']
datesdrop=['2018-11-01','2018-12-01','2019-01-01','2019-02-01','2019-03-01',
         '2020-01-01','2020-02-01','2020-03-01','2020-04-01','2020-05-01','2020-06-01','2020-07-01',
         '2020-08-01','2020-09-01','2020-10-01','2020-11-01','2020-12-01']

collist=[]
droplist=[]
for col in datesall:
    date=pd.to_datetime(col)
    collist.append(date)
for col in datesdrop:
    date=pd.to_datetime(col)
    droplist.append(date)
    

colpd=pd.DataFrame({'Date':collist})
print(colpd)

         Date
0  2018-11-01
1  2018-12-01
2  2019-01-01
3  2019-02-01
4  2019-03-01
5  2019-04-01
6  2019-05-01
7  2019-06-01
8  2019-07-01
9  2019-08-01
10 2019-09-01
11 2019-10-01
12 2019-11-01
13 2019-12-01
14 2020-01-01
15 2020-02-01
16 2020-03-01
17 2020-04-01
18 2020-05-01
19 2020-06-01
20 2020-07-01
21 2020-08-01
22 2020-09-01
23 2020-10-01
24 2020-11-01
25 2020-12-01


In [5]:
bclients='/Users/JoyThompson/Desktop/Automate CK Monthly Billing/brianclients.xlsx'
sclients='/Users/JoyThompson/Desktop/Automate CK Monthly Billing/spencerclients.xlsx'
pclients='/Users/JoyThompson/Desktop/Automate CK Monthly Billing/phelanclients.xlsx'
briclients= pd.ExcelFile(bclients)
print(briclients)
spenclients= pd.ExcelFile(sclients)
brisheets=briclients.sheet_names
spencsheets=spenclients.sheet_names
phaclients= pd.ExcelFile(pclients)
phalensheets=phaclients.sheet_names
phalensheets=['geckobrands', 'C4 Belts',
               'Marula Oil', 'geckobrands', 'livBay Lash', 'P.E. Nation' ]
print(phalensheets)

['geckobrands', 'C4 Belts', 'Marula Oil', 'geckobrands', 'livBay Lash', 'P.E. Nation']


In [6]:
clientdf=pd.DataFrame(columns=collist, index=clients)

for ind in clientdf.index:
    print(ind)
    if ind in brisheets:
        dfclient=pd.read_excel(briclients, sheetname=ind)
           
        for count in range(33, 45):
            
            invoice=dfclient.iloc[count, 5]

            if str(invoice)[0]!='n' and str(invoice)[0]!='I':
                dateorig=dfclient.iloc[count, 0]
                print(dateorig)
                
                index=[i for i, x in enumerate(colpd['Date']==dateorig) if x]
                print(index)
                indexdate=index[0]-1
                print(indexdate)
                date=colpd.loc[indexdate, 'Date']
                
                clientdf.loc[ind, date]=np.round(float(invoice), decimals=2)

    if ind in spencsheets:
        dfclient=pd.read_excel(spenclients, sheetname=ind)


        for count in range(33, 45):

            invoice=dfclient.iloc[count, 5]
   
            if str(invoice)[0]!='n' and str(invoice)[0]!='I':
                dateorig=dfclient.iloc[count, 0]
                
                index=[i for i, x in enumerate(colpd['Date']==dateorig) if x]

                indexdate=index[0]-1
            
                date=colpd.loc[indexdate, 'Date']
  
                clientdf.loc[ind, date]=np.round(float(invoice), decimals=2)
           

    if ind in phalensheets:
        dfclient=pd.read_excel(pclients, sheetname=ind)


        for count in range(33, 45):

            invoice=dfclient.iloc[count, 5]
    
            if str(invoice)[0] not in ['n', 'P', 'I', 'V']:
                dateorig=dfclient.iloc[count, 0]
                
                index=[i for i, x in enumerate(colpd['Date']==dateorig) if x]
        
                indexdate=index[0]-1
                date=colpd.loc[indexdate, 'Date']
        
                clientdf.loc[ind, date]=np.round(float(invoice), decimals=2)

    if ind == 'Outdoor Hub (Five Ten)':
         for count in outfc.index:

            date=outfc.loc[count, 'Date']

            invoice=outfc.loc[count, 'Billing']
            clientdf.loc[ind, date]=invoice
        

3B Medical
2019-05-01 00:00:00
[6]
5
2019-06-01 00:00:00
[7]
6
2019-07-01 00:00:00
[8]
7
2019-08-01 00:00:00
[9]
8
2019-09-01 00:00:00
[10]
9
2019-10-01 00:00:00
[11]
10
2019-11-01 00:00:00
[12]
11
2019-12-01 00:00:00
[13]
12
2020-01-01 00:00:00
[14]
13
Ambreesh
2019-09-01 00:00:00
[10]
9
2019-10-01 00:00:00
[11]
10
2019-11-01 00:00:00
[12]
11
2019-12-01 00:00:00
[13]
12
2020-01-01 00:00:00
[14]
13
2020-02-01 00:00:00
[15]
14
2020-03-01 00:00:00
[16]
15
2020-04-01 00:00:00
[17]
16
2020-05-01 00:00:00
[18]
17
2020-06-01 00:00:00
[19]
18
2020-07-01 00:00:00
[20]
19
2020-08-01 00:00:00
[21]
20
BHF-USA
2019-08-01 00:00:00
[9]
8
2019-09-01 00:00:00
[10]
9
2019-10-01 00:00:00
[11]
10
2019-11-01 00:00:00
[12]
11
2019-12-01 00:00:00
[13]
12
2020-01-01 00:00:00
[14]
13
2020-02-01 00:00:00
[15]
14
2020-03-01 00:00:00
[16]
15
2020-04-01 00:00:00
[17]
16
2020-05-01 00:00:00
[18]
17
2020-06-01 00:00:00
[19]
18
2020-07-01 00:00:00
[20]
19
Brain Armor
2019-05-01 00:00:00
[6]
5
2019-06-01 00:00:00
[7]

In [7]:
monthvec=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September','October', 'November', 'December']
clientdf2=clientdf.drop(columns=droplist, axis=0)
for col in clientdf2.columns:
    year=str(col)[0:4]
    monthnum=str(col)[5:7]
    monthnum=int(monthnum)-1
    month=monthvec[monthnum]
    newname=month+ ' ' + year
    clientdf2=clientdf2.rename(columns={col:newname})
print(clientdf2)



                                April 2019 May 2019 June 2019 July 2019  \
Client Name                                                               
3B Medical                         9671.28  16406.1   16241.5   19212.3   
Ambreesh                               NaN      NaN       NaN       NaN   
BHF-USA                                NaN      NaN       NaN      2000   
Brain Armor                           5000     3000      3000      3000   
C4 Belts                               NaN      NaN       NaN       NaN   
Coyuchi                               5000     3000      3000      3000   
Dammit Dolls                        2529.1   2373.5   2510.75   2473.35   
DaVinci                               5000     5000      5000         0   
Dedeco                                3000     3000      3000      3000   
Domesis                            11708.8  9976.18   7267.01   7924.27   
Fast Growing Trees                     NaN      NaN       NaN       NaN   
Outdoor Hub (Five Ten)   

In [8]:
clientdf2=clientdf2.reset_index()
clientdf2['Client Total Billing']=0
for ind in clientdf2.index:
    totsum=np.sum(clientdf2.loc[ind, :])
    clientdf2.loc[ind, 'Client Total Billing']=float(totsum)
print(clientdf2)


TypeError: can only concatenate str (not "numpy.float64") to str

In [9]:
sumlist=[]
for col in clientdf2.columns:
    totsum=np.sum(clientdf2.loc[:, col])
    sumlist.append(float(totsum))
clientdf2.loc['Monthly Total', :]=sumlist


ValueError: could not convert string to float: "3B MedicalAmbreeshBHF-USABrain ArmorC4 BeltsCoyuchiDammit DollsDaVinciDedecoDomesisFast Growing TreesOutdoor Hub (Five Ten)Forever NewgeckobrandsGeoxGinsuIndosoleInnovaJax WaxJuggernautLittle GiraffeLivBay Lash (sassy lashes)Marula OilMasqueBarMaviMolexMontana WestNano FilmNot RatedOdd SoxOrganycP.E. NationPerry EllisPet 'n ShapePeter GrimmRagoSnif SaxSoludosSpin LifeWestmoorWine EnthusiastWorkman's FriendWorldbookDavid's WholesaleKnit Rite (Therafirm)Knit Rite (Therafirm) - WalmartStreet StriderOutdoor Hub (Five Ten)"

In [10]:
print(clientdf2)

                        Client Name April 2019 May 2019 June 2019 July 2019  \
0                        3B Medical    9671.28  16406.1   16241.5   19212.3   
1                          Ambreesh        NaN      NaN       NaN       NaN   
2                           BHF-USA        NaN      NaN       NaN      2000   
3                       Brain Armor       5000     3000      3000      3000   
4                          C4 Belts        NaN      NaN       NaN       NaN   
5                           Coyuchi       5000     3000      3000      3000   
6                      Dammit Dolls     2529.1   2373.5   2510.75   2473.35   
7                           DaVinci       5000     5000      5000         0   
8                            Dedeco       3000     3000      3000      3000   
9                           Domesis    11708.8  9976.18   7267.01   7924.27   
10               Fast Growing Trees        NaN      NaN       NaN       NaN   
11           Outdoor Hub (Five Ten)    7374.88  8560

In [11]:
filenamefinal = '/Users/JoyThompson/Desktop/Clientbillingauto9.xlsx'
writer = pd.ExcelWriter(filenamefinal, engine='xlsxwriter')
clientdf2.to_excel(writer, sheet_name='Billing Auto')
writer.save()